In [78]:
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split, Dataset
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [79]:
import os

class SaveData:
    def __init__(self, architecture=[],hyperparameter=[],encoder=None,decoder=None,plots=None, folder_name=None, relative_path="./data"):
        self.architecture = architecture
        self.hyperparameter = hyperparameter
        self.encoder = encoder
        self.decoder = decoder
        self.plots = plots
        self.folder_name = folder_name
        self.relative_path = relative_path
    
    def save_data_to_txt(self,save_architecture=False):
        # If dict doesn't exist, create new folder
        if self.folder_name:
            directory = os.path.join(self.relative_path, self.folder_name)
            if not os.path.exists(directory):
                os.makedirs(directory)
        else:
            directory = self.relative_path

        # Create a new .txt at the directory
        file_name = "data.txt"
        file_path = os.path.join(directory, file_name)

        with open(file_path, "w") as f:
            # Write each item in the list to a new line in the file
            f.write(str("Architecture:")+ "\n" + "\n")
            for item in self.architecture:
                f.write(str(item) + "\n")
            f.write(str("Hyperparameters:")+ "\n" + "\n")
            for item in self.hyperparameter:
                f.write(str(item) + "\n")

        if save_architecture==True:
            file_name = ["decoder.pt","encoder.pt"]
            file_path = [os.path.join(directory, file_name[0]),os.path.join(directory, file_name[1])]
            torch.save(self.encoder.state_dict(), file_path[0])
            torch.save(self.decoder.state_dict(), file_path[1])

        
        print(f"Architecture and hyperparameters saved to {file_path}.txt")

    def save_plots(self):
        if self.folder_name:
            directory = os.path.join(self.relative_path, self.folder_name)
            if not os.path.exists(directory):
                os.makedirs(directory)
        else:
            directory = self.relative_path

        # Create a new .txt at the directory
        file_name = "plot.png"
        file_path = os.path.join(directory, file_name)

        plt.savefig(file_path)
        plt.close()

        print(f"plot saved to {file_path}.png")
        return ##This is under revision as of what is smartest
        
    


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, encoded_space_dim, fc2_input_dim):
        super().__init__()
        self.encoded_space_dim = encoded_space_dim
        self.fc2_input_dim = fc2_input_dim
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(in_channels=35, out_channels=128, kernel_size=(1, 4), stride=1, padding=(0, 1)), #(1,255)
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=(1, 2), padding=0), #(1,127)
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(1, 4), stride=1, padding=(0, 1)), #(1,126)
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=(1, 2), padding=0), #(1,63)
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(1, 4), stride=1, padding=(0, 1)), #(1,62)
            nn.Tanh(),
        )
        self.flatten = nn.Flatten(start_dim=1)
        self.encoder_lin = nn.Sequential(
            nn.Linear(7936, 512),
            nn.Tanh(),
            nn.Linear(512, self.encoded_space_dim)
        )
        self.decoder_lin = nn.Sequential(
            nn.Linear(self.encoded_space_dim, 512),
            nn.Tanh(),
            nn.Linear(512, 7936),
            nn.Tanh()
        )
        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(128, 1, 62))
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(1, 4), stride=1, padding=(0, 1), output_padding=0), #(1,63)
            nn.Tanh(),
            nn.Upsample(scale_factor=2, mode='bilinear'), #(1,126)
            nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(1, 4), stride=1, padding=(0, 1), output_padding=0), #(1,127)
            nn.Tanh(),
            nn.Upsample(size=(1, 255), mode='bilinear'), #(1,255)
            nn.ConvTranspose2d(in_channels=128, out_channels=35, kernel_size=(1, 4), stride=1, padding=(0, 1), output_padding=0), #(1,256)
            nn.Tanh()
        )
    
    def encode(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        return x
    
    def decode(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x
    
    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x

In [80]:
class Encoder1(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(in_channels=35, out_channels=128, kernel_size=(1,4), stride=1, padding=(0,1)), #(1,255)
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=(1,2), padding=0), #(1,127)
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(1,4), stride=1, padding=(0,1)), #(1,126)
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=(1,2), padding=0), #(1,63)
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(1,4), stride=1, padding=(0,1)), #(1,62)
            nn.Tanh(),
            #nn.MaxPool2d(kernel_size=(1,2), padding=0), #(1,27)
            #nn.Conv2d(in_channels=100, out_channels=100, kernel_size=(1,8), stride=1, padding=(0,1)), #(1,22)
            #nn.Tanh()
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(7936, 512),
            nn.Tanh(),
            nn.Linear(512, encoded_space_dim)
        )
        
    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        return x
class Decoder2(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 512),
            nn.Tanh(),
            nn.Linear(512, 7936),
            nn.Tanh()
        )

        self.unflatten = nn.Unflatten(dim=1, 
        unflattened_size=(128, 1, 62))

        self.decoder_conv = nn.Sequential(
            #nn.ConvTranspose2d(in_channels=100, out_channels=100, kernel_size=(1,8), stride=1, padding=(0,1), output_padding=0), #(1,27)
            #nn.Tanh(),
            #nn.Upsample(size=(1,55), mode='bilinear'), #(1,55)
            nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(1,4), stride=1, padding=(0,1), output_padding=0), #(1,63)
            nn.Tanh(),
            nn.Upsample(scale_factor=2, mode='bilinear'), #(1,126)
            nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(1,4), stride=1, padding=(0,1), output_padding=0), #(1,127)
            nn.Tanh(),
            nn.Upsample(size=(1,255), mode='bilinear'),#(1,255)
            nn.ConvTranspose2d(in_channels=128, out_channels=35, kernel_size=(1,4), stride=1, padding=(0,1), output_padding=0), #(1,256)
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [81]:
### Define the loss function
loss_fn = torch.nn.MSELoss()

#loss_fn = torch.nn.L1Loss()

### Define an optimizer (both for the encoder and the decoder!)
lr= 0.01

### Set the random seed for reproducible results
torch.manual_seed(0)

### Initialize the two networks
d = 64

#model = Autoencoder(encoded_space_dim=encoded_space_dim)
encoder = Encoder1(encoded_space_dim=d,fc2_input_dim=35)
decoder = Decoder2(encoded_space_dim=d,fc2_input_dim=35)
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

#torch.save(encoder.state_dict(), 'encoder_model.pt')
#torch.save(decoder.state_dict(), 'decoder_model.pt')

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)
#optim = torch.optim.SGD(params_to_optimize, lr=lr, weight_decay=1e-05,momentum=0.9)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
encoder.to(device)
decoder.to(device)
architecture = [encoder.to(device),decoder.to(device)]

Selected device: cuda


In [86]:
my_data = architecture
hyperparameter = ["lr=5","loss=MSE"]
data_saver = SaveData(architecture=my_data, hyperparameter=hyperparameter,encoder=encoder,decoder=decoder, folder_name="Data_yolo", relative_path="./data_files")
data_saver.save_data_to_txt(save_architecture=True)


Architecture and hyperparameters saved to ['./data_files\\Data_yolo\\decoder.pt', './data_files\\Data_yolo\\encoder.pt'].txt


In [92]:
PATH = "./data_files/Data_yolo/decoder.pt"

model = torch.load(PATH)
model.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'